# Notebook 6 · Feature Mart & Label Engineering

This workbook consolidates Apple Health and Libre glucose features into reusable daily and event-level datasets, setting the stage for wellness, cardio, strength, and stress recommendations.

**Objectives**
- Reuse existing SQLite tables and processed CSV exports without re-parsing source XML/CSV files
- Load daily Apple Health aggregates and glucose metrics into a unified "feature mart"
- Prepare scaffolding for event-centric features (meals, cardio, strength, stress/readiness)
- Persist interim datasets for downstream modeling notebooks

**Notebook roadmap**
1. Configure environment and helper utilities
2. Inspect SQLite inventory and load core tables
3. Load processed CSV feature exports
4. Build the first pass of the daily feature mart and persist to Parquet
5. Scaffold event-centric feature pipelines and TODOs

In [48]:
# Imports and display defaults
from __future__ import annotations

from pathlib import Path
import sqlite3

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 3)
print("✅ Libraries imported")

✅ Libraries imported


In [49]:
# Project paths and database connection helpers
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
DB_PATH = DATA_DIR / "database" / "health_glucose.db"

assert DB_PATH.exists(), f"SQLite database missing at {DB_PATH}"


def get_connection(db_path: Path) -> sqlite3.Connection:
    """Create a read-only SQLite connection when possible."""
    uri = f"file:{db_path}?mode=ro"
    try:
        return sqlite3.connect(uri, uri=True)
    except sqlite3.OperationalError:
        return sqlite3.connect(db_path)


conn = get_connection(DB_PATH)
print(f"📀 Connected to SQLite at {DB_PATH}")

📀 Connected to SQLite at /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/data/database/health_glucose.db


In [50]:
# Inspect available tables and basic row counts
def list_tables(connection: sqlite3.Connection) -> pd.DataFrame:
    query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
    return pd.read_sql(query, connection)


def table_counts(
    connection: sqlite3.Connection, table_names: list[str]
) -> pd.DataFrame:
    counts = []
    for name in table_names:
        try:
            row_count = pd.read_sql(
                f"SELECT COUNT(*) AS row_count FROM {name};", connection
            )["row_count"].iat[0]
        except Exception as exc:
            row_count = np.nan
            print(f"⚠️ Could not count table {name}: {exc}")
        counts.append({"table": name, "row_count": row_count})
    return pd.DataFrame(counts)


table_inventory = list_tables(conn)
display(Markdown("**SQLite tables available:**"))
display(table_inventory)

core_tables = ["glucose_readings", "apple_health_records", "windowed_glucose_features"]
existing_tables = [t for t in core_tables if t in table_inventory["name"].tolist()]
display(Markdown("**Row counts for core tables:**"))
display(table_counts(conn, existing_tables))

**SQLite tables available:**

,name
0,apple_health_records
1,data_quality_log
2,glucose_readings
3,glucose_statistics
4,merged_health_data
5,sqlite_sequence
6,workout_records


**Row counts for core tables:**

,table,row_count
0,glucose_readings,4151
1,apple_health_records,114252


In [51]:
# Load core SQLite tables into DataFrames (subset to necessary columns for efficiency)
GLUCOSE_COLUMNS = [
    "timestamp",
    "glucose_value",
    "glucose_range",
    "glucose_rate_change",
    "glucose_trend",
]
WINDOW_COLUMNS = [
    "window_start",
    "window_end",
    "mean_glucose",
    "std_glucose",
    "min_glucose",
    "max_glucose",
]

glucose_query = f"SELECT {', '.join(GLUCOSE_COLUMNS)} FROM glucose_readings;"
glucose_df = pd.read_sql(glucose_query, conn, parse_dates=["timestamp"])
print(f"✅ Loaded glucose_readings: {len(glucose_df):,} rows")

apple_records_df = pd.read_sql("SELECT * FROM apple_health_records;", conn)
date_like_cols = [
    col
    for col in apple_records_df.columns
    if col.lower().endswith("date") or col.lower().endswith("time")
]
for col in date_like_cols:
    apple_records_df[col] = pd.to_datetime(apple_records_df[col], errors="coerce")
print(f"✅ Loaded apple_health_records: {len(apple_records_df):,} rows")

if "windowed_glucose_features" in existing_tables:
    window_query = f"SELECT {', '.join(WINDOW_COLUMNS)} FROM windowed_glucose_features;"
    windowed_glucose_df = pd.read_sql(
        window_query, conn, parse_dates=["window_start", "window_end"]
    )
    print(f"✅ Loaded windowed_glucose_features: {len(windowed_glucose_df):,} rows")
else:
    windowed_glucose_df = pd.DataFrame()
    print("ℹ️ Table windowed_glucose_features not found; continuing without it.")

✅ Loaded glucose_readings: 4,151 rows
✅ Loaded apple_health_records: 114,252 rows
ℹ️ Table windowed_glucose_features not found; continuing without it.
✅ Loaded apple_health_records: 114,252 rows
ℹ️ Table windowed_glucose_features not found; continuing without it.


In [52]:
# Load processed CSV feature exports for Apple Health daily metrics
APPLE_DAILY_PATH = PROCESSED_DIR / "apple_health_daily_features.csv"
APPLE_SUMMARY_PATH = PROCESSED_DIR / "apple_health_type_summary.csv"
MERGED_HEALTH_GLUCOSE_PATH = PROCESSED_DIR / "merged_health_glucose.csv"


def safe_read_csv(
    path: Path,
    parse_dates: list[str] | None = None,
    infer_datetime: bool = True,
    **kwargs,
) -> pd.DataFrame:
    if not path.exists():
        print(f"⚠️ Missing CSV: {path.name}")
        return pd.DataFrame()
    df = pd.read_csv(path, **kwargs)
    candidate_cols: set[str] = set(parse_dates or [])
    if infer_datetime:
        inferred = {
            col
            for col in df.columns
            if "timestamp" in col.lower() or col.lower().endswith("date")
        }
        candidate_cols.update(inferred)
    for col in sorted(candidate_cols):
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
        else:
            print(
                f"⚠️ Column '{col}' not found in {path.name}; skipping datetime parsing.",
            )
    print(f"📄 Loaded {path.name}: {len(df):,} rows")
    return df


apple_daily_df = safe_read_csv(
    APPLE_DAILY_PATH, parse_dates=["event_date"], infer_datetime=True
)
apple_summary_df = safe_read_csv(APPLE_SUMMARY_PATH, infer_datetime=False)
merged_health_glucose_df = safe_read_csv(
    MERGED_HEALTH_GLUCOSE_PATH,
    parse_dates=["glucose_timestamp", "health_timestamp", "creationDate", "endDate"],
    infer_datetime=True,
)

📄 Loaded apple_health_daily_features.csv: 15 rows
📄 Loaded apple_health_type_summary.csv: 40 rows
📄 Loaded merged_health_glucose.csv: 114,252 rows
📄 Loaded merged_health_glucose.csv: 114,252 rows


In [53]:
# Inspect Apple daily and merged health schemas to guide feature engineering
display(Markdown("**Apple daily features schema preview**"))
display(apple_daily_df.head())
print("Apple daily dtypes:")
print(apple_daily_df.dtypes.head())

display(Markdown("**Merged health + glucose schema preview**"))
display(merged_health_glucose_df.head())
print("Merged health + glucose dtypes (subset):")
print(
    merged_health_glucose_df[
        ["glucose_timestamp", "health_timestamp", "creationDate", "endDate"]
    ].dtypes
)

**Apple daily features schema preview**

,event_date,mean_ActiveEnergyBurned,mean_AppleExerciseTime,mean_AppleStandTime,mean_AppleWalkingSteadiness,mean_BasalEnergyBurned,mean_BloodGlucose,mean_BodyFatPercentage,mean_BodyMass,mean_BodyMassIndex,mean_DietaryCarbohydrates,mean_DietaryCholesterol,mean_DietaryEnergyConsumed,mean_DietaryFatSaturated,mean_DietaryFatTotal,mean_DietaryFiber,mean_DietaryPotassium,mean_DietaryProtein,mean_DietarySodium,mean_DietarySugar,mean_DietaryWater,mean_DistanceCycling,mean_DistanceWalkingRunning,mean_EnvironmentalAudioExposure,mean_EnvironmentalSoundReduction,mean_FlightsClimbed,mean_HeadphoneAudioExposure,mean_HeartRate,mean_HeartRateVariabilitySDNN,mean_LeanBodyMass,mean_PhysicalEffort,mean_RespiratoryRate,mean_RestingHeartRate,mean_SixMinuteWalkTestDistance,mean_StepCount,mean_VO2Max,mean_WalkingAsymmetryPercentage,mean_WalkingDoubleSupportPercentage,mean_WalkingHeartRateAverage,mean_WalkingSpeed,mean_WalkingStepLength,sum_ActiveEnergyBurned,sum_AppleExerciseTime,sum_AppleStandTime,sum_AppleWalkingSteadiness,sum_BasalEnergyBurned,sum_BloodGlucose,sum_BodyFatPercentage,sum_BodyMass,sum_BodyMassIndex,sum_DietaryCarbohydrates,sum_DietaryCholesterol,sum_DietaryEnergyConsumed,sum_DietaryFatSaturated,sum_DietaryFatTotal,sum_DietaryFiber,sum_DietaryPotassium,sum_DietaryProtein,sum_DietarySodium,sum_DietarySugar,sum_DietaryWater,sum_DistanceCycling,sum_DistanceWalkingRunning,sum_EnvironmentalAudioExposure,sum_EnvironmentalSoundReduction,sum_FlightsClimbed,sum_HeadphoneAudioExposure,sum_HeartRate,sum_HeartRateVariabilitySDNN,sum_LeanBodyMass,sum_PhysicalEffort,sum_RespiratoryRate,sum_RestingHeartRate,sum_SixMinuteWalkTestDistance,sum_StepCount,sum_VO2Max,sum_WalkingAsymmetryPercentage,sum_WalkingDoubleSupportPercentage,sum_WalkingHeartRateAverage,sum_WalkingSpeed,sum_WalkingStepLength,count_ActiveEnergyBurned,count_AppleExerciseTime,count_AppleStandTime,count_AppleWalkingSteadiness,count_BasalEnergyBurned,count_BloodGlucose,count_BodyFatPercentage,count_BodyMass,count_BodyMassIndex,count_DietaryCarbohydrates,count_DietaryCholesterol,count_DietaryEnergyConsumed,count_DietaryFatSaturated,count_DietaryFatTotal,count_DietaryFiber,count_DietaryPotassium,count_DietaryProtein,count_DietarySodium,count_DietarySugar,count_DietaryWater,count_DistanceCycling,count_DistanceWalkingRunning,count_EnvironmentalAudioExposure,count_EnvironmentalSoundReduction,count_FlightsClimbed,count_HeadphoneAudioExposure,count_HeartRate,count_HeartRateVariabilitySDNN,count_LeanBodyMass,count_PhysicalEffort,count_RespiratoryRate,count_RestingHeartRate,count_SixMinuteWalkTestDistance,count_StepCount,count_VO2Max,count_WalkingAsymmetryPercentage,count_WalkingDoubleSupportPercentage,count_WalkingHeartRateAverage,count_WalkingSpeed,count_WalkingStepLength
0,2025-08-16,0.486,1.0,2.132,NaN,0.576,103.898,NaN,NaN,NaN,6.500,79.101,121.143,1.981,6.400,2.695,723.265,19.000,73.824,3.587,NaN,NaN,0.061,60.936,16.526,2.556,71.563,97.021,31.129,NaN,3.753,NaN,NaN,NaN,86.107,NaN,0.015,0.271,NaN,2.877,29.497,917.858,153.0,145.0,NaN,857.592,13299.0,NaN,NaN,NaN,26.0,237.304,848.0,5.943,32.0,8.085,3616.327,95.0,442.946,14.349,NaN,NaN,5.871,974.969,66.102,23.0,1216.569,136216.911,124.516,NaN,1460.1,NaN,NaN,NaN,12055.0,NaN,0.16,5.965,NaN,74.803,766.930,1887.0,153.0,68.0,NaN,1489.0,128.0,NaN,NaN,NaN,4.0,3.0,7.0,3.0,5.0,3.0,5.0,5.0,6.0,4.0,NaN,NaN,97.0,16.0,4.0,9.0,17.0,1404.0,4.0,NaN,389.0,NaN,NaN,NaN,140.0,NaN,11.0,22.0,NaN,26.0,26.0
1,2025-08-17,0.655,1.0,2.348,NaN,2.134,91.795,0.143,153.102,22.654,17.444,122.688,118.900,2.087,8.667,6.771,321.571,8.667,21.079,2.644,0.0,NaN,0.047,50.569,16.856,2.667,76.450,82.268,36.536,131.087,3.850,14.327,58.0,500.0,124.781,NaN,0.001,0.275,79.5,2.780,29.209,869.902,59.0,155.0,NaN,1813.948,13402.0,0.143,153.102,45.309,157.0,245.375,1189.0,14.611,52.0,33.857,1286.285,52.0,147.551,23.793,0.0,NaN,6.973,2022.749,50.569,16.0,229.351,184362.608,328.822,131.087,1594.1,7865.5,58.0,500.0,14225.0,NaN,0.01,5.216,79.5,58.384,613.386,1329.0,59.0,66.0,NaN,850.0,146.0,1.0,1.0,2

Apple daily dtypes:
event_date                     datetime64[ns]
mean_ActiveEnergyBurned               float64
mean_AppleExerciseTime                float64
mean_AppleStandTime                   float64
mean_AppleWalkingSteadiness           float64
dtype: object


**Merged health + glucose schema preview**

,type,sourceName,value,unit,creationDate,health_timestamp,endDate,device,serial_number,glucose_timestamp,record_type,glucose_mg_dl,scan_glucose_mg_dl,Non-numeric Rapid-Acting Insulin,rapid_acting_insulin_units,Non-numeric Food,carbohydrates_grams,Carbohydrates (servings),Non-numeric Long-Acting Insulin,long_acting_insulin_units,notes,strip_glucose_mg_dl,ketone_mmol_l,Meal Insulin (units),Correction Insulin (units),User Change Insulin (units),glucose_value,glucose_source,glucose_rate_change,glucose_trend,hour,day_of_week,is_weekend,glucose_range,time_diff_minutes,is_night,is_morning,is_afternoon,is_evening,likely_meal_time
0,HKQuantityTypeIdentifierHeartRate,Zepp,78.000,count/min,2025-08-17 00:48:15,2025-08-16 13:20:00,2025-08-16 13:20:59,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.000,False,False,True,False,True
1,HKQuantityTypeIdentifierAppleStandTime,George’s Apple Watch,3.000,min,2025-08-16 13:25:35,2025-08-16 13:20:00,2025-08-16 13:25:00,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.000,False,False,True,False,True
2,HKQuantityTypeIdentifierActiveEnergyBurned,Zepp,1.000,Cal,2025-09-19 22:50:35,2025-08-16 13:20:00,2025-08-16 13:29:59,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.000,False,False,True,False,True
3,HKQuantityTypeIdentifierBloodGlucose,Bevel,125.000,mg/dL,2025-08-17 00:38:00,2025-08-16 13:20:01,2025-08-16 13:20:01,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.017,False,False,True,False,True
4,HKQuantityTypeIdentifierActiveEnergyBurned,George’s Apple Watch,0.465,Cal,2025-08-16 13:21:13,2025-08-16 13:20:12,2025-08-16 13:20:53,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.200,False,False,True,False,True


Merged health + glucose dtypes (subset):
glucose_timestamp    datetime64[ns]
health_timestamp     datetime64[ns]
creationDate         datetime64[ns]
endDate              datetime64[ns]
dtype: object


In [54]:
merged_health_glucose_df.head()

,type,sourceName,value,unit,creationDate,health_timestamp,endDate,device,serial_number,glucose_timestamp,record_type,glucose_mg_dl,scan_glucose_mg_dl,Non-numeric Rapid-Acting Insulin,rapid_acting_insulin_units,Non-numeric Food,carbohydrates_grams,Carbohydrates (servings),Non-numeric Long-Acting Insulin,long_acting_insulin_units,notes,strip_glucose_mg_dl,ketone_mmol_l,Meal Insulin (units),Correction Insulin (units),User Change Insulin (units),glucose_value,glucose_source,glucose_rate_change,glucose_trend,hour,day_of_week,is_weekend,glucose_range,time_diff_minutes,is_night,is_morning,is_afternoon,is_evening,likely_meal_time
0,HKQuantityTypeIdentifierHeartRate,Zepp,78.000,count/min,2025-08-17 00:48:15,2025-08-16 13:20:00,2025-08-16 13:20:59,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.000,False,False,True,False,True
1,HKQuantityTypeIdentifierAppleStandTime,George’s Apple Watch,3.000,min,2025-08-16 13:25:35,2025-08-16 13:20:00,2025-08-16 13:25:00,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.000,False,False,True,False,True
2,HKQuantityTypeIdentifierActiveEnergyBurned,Zepp,1.000,Cal,2025-09-19 22:50:35,2025-08-16 13:20:00,2025-08-16 13:29:59,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.000,False,False,True,False,True
3,HKQuantityTypeIdentifierBloodGlucose,Bevel,125.000,mg/dL,2025-08-17 00:38:00,2025-08-16 13:20:01,2025-08-16 13:20:01,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.017,False,False,True,False,True
4,HKQuantityTypeIdentifierActiveEnergyBurned,George’s Apple Watch,0.465,Cal,2025-08-16 13:21:13,2025-08-16 13:20:12,2025-08-16 13:20:53,FreeStyle Libre 3,03EE5610-75AC-4A63-860F-A3A70A27AD2F,2025-08-16 13:20:00,0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,historic,NaN,stable,13,5,True,normal,0.200,False,False,True,False,True


In [55]:
def compute_sleep_features(apple_records: pd.DataFrame) -> pd.DataFrame:
    """Aggregate Apple Sleep Analysis records into daily sleep metrics."""
    columns = [
        "event_date",
        "sleep_duration_hours",
        "sleep_asleep_hours",
        "sleep_inbed_hours",
        "sleep_sessions",
        "sleep_efficiency_pct",
        "sleep_onset_time",
        "sleep_wake_time",
        "sleep_midpoint_time",
        "sleep_latency_minutes",
        "sleep_restful_ratio_pct",
    ]
    if apple_records.empty or "type" not in apple_records.columns:
        return pd.DataFrame(columns=columns)
    sleep_df = apple_records[
        apple_records["type"].str.contains("sleep", case=False, na=False)
    ].copy()
    if sleep_df.empty:
        return pd.DataFrame(columns=columns)

    def _resolve_column(df: pd.DataFrame, candidates: list[str]) -> str | None:
        lower_map = {col.lower(): col for col in df.columns}
        for candidate in candidates:
            if candidate in df.columns:
                return candidate
            lowered = candidate.lower()
            if lowered in lower_map:
                return lower_map[lowered]
        for col in df.columns:
            if any(candidate.lower() in col.lower() for candidate in candidates):
                return col
        return None

    start_col = _resolve_column(
        sleep_df, ["start_date", "startDate", "start_time", "startTime"]
    )
    end_col = _resolve_column(sleep_df, ["end_date", "endDate", "end_time", "endTime"])
    if start_col is None or end_col is None:
        return pd.DataFrame(columns=columns)

    sleep_df["start_ts"] = pd.to_datetime(sleep_df[start_col], errors="coerce")
    sleep_df["end_ts"] = pd.to_datetime(sleep_df[end_col], errors="coerce")
    sleep_df = sleep_df.dropna(subset=["start_ts", "end_ts"]).sort_values("start_ts")
    if sleep_df.empty:
        return pd.DataFrame(columns=columns)

    value_col = _resolve_column(
        sleep_df, ["value", "sleep_state", "sleepStage", "sleep_stage"]
    )
    if value_col:
        sleep_df["value_str"] = sleep_df[value_col].astype(str).str.lower()
    else:
        sleep_df["value_str"] = ""

    sleep_df["duration_hours"] = (
        sleep_df["end_ts"] - sleep_df["start_ts"]
    ).dt.total_seconds() / 3600
    sleep_df["event_date"] = sleep_df["end_ts"].dt.floor("D")

    def _summarize(group: pd.DataFrame) -> pd.Series:
        event_date = pd.Timestamp(group.name).floor("D")
        asleep_mask = group["value_str"].str.contains(
            "asleep|core|deep|rem", regex=True, na=False
        )
        inbed_mask = group["value_str"].str.contains(
            "inbed|in bed|bed", regex=True, na=False
        )
        restful_mask = group["value_str"].str.contains("deep|rem", regex=True, na=False)
        total_hours = group["duration_hours"].sum()
        asleep_hours = group.loc[asleep_mask, "duration_hours"].sum()
        inbed_hours = group.loc[inbed_mask, "duration_hours"].sum()
        if asleep_hours == 0:
            asleep_hours = total_hours
        if inbed_hours == 0:
            inbed_hours = total_hours
        sleep_onset = group["start_ts"].min()
        wake_time = group["end_ts"].max()
        midpoint = (
            sleep_onset + (wake_time - sleep_onset) / 2
            if pd.notna(sleep_onset) and pd.notna(wake_time)
            else pd.NaT
        )
        sleep_latency = (
            (sleep_onset - event_date).total_seconds() / 60
            if pd.notna(sleep_onset)
            else np.nan
        )
        restful_hours = group.loc[restful_mask, "duration_hours"].sum()
        return pd.Series(
            {
                "event_date": event_date,
                "sleep_duration_hours": total_hours,
                "sleep_asleep_hours": asleep_hours,
                "sleep_inbed_hours": inbed_hours,
                "sleep_sessions": group.shape[0],
                "sleep_efficiency_pct": (
                    (asleep_hours / inbed_hours * 100) if inbed_hours else np.nan
                ),
                "sleep_onset_time": sleep_onset,
                "sleep_wake_time": wake_time,
                "sleep_midpoint_time": midpoint,
                "sleep_latency_minutes": sleep_latency,
                "sleep_restful_ratio_pct": (
                    (restful_hours / asleep_hours * 100) if asleep_hours else np.nan
                ),
            }
        )

    summary = (
        sleep_df.groupby("event_date", group_keys=False)
        .apply(_summarize, include_groups=False)
        .reset_index(drop=True)
    )
    summary["event_date"] = pd.to_datetime(summary["event_date"]).dt.tz_localize(None)
    return summary[columns]


def _add_baseline_columns(
    df: pd.DataFrame, column: str, baseline_col: str, delta_col: str, window: int
) -> None:
    if column in df.columns:
        df[baseline_col] = df[column].rolling(window=window, min_periods=3).mean()
        df[delta_col] = df[column] - df[baseline_col]


def add_trailing_baselines(daily_df: pd.DataFrame) -> pd.DataFrame:
    """Compute trailing baselines and deltas for key wellness metrics."""
    if daily_df.empty or "date" not in daily_df.columns:
        return daily_df
    df = daily_df.sort_values("date").copy()
    _add_baseline_columns(
        df,
        "mean_RestingHeartRate",
        "resting_hr_baseline_14d",
        "resting_hr_delta",
        window=14,
    )
    _add_baseline_columns(
        df, "mean_HeartRateVariabilitySDNN", "hrv_baseline_14d", "hrv_delta", window=14
    )
    _add_baseline_columns(
        df, "mean_VO2Max", "vo2max_baseline_28d", "vo2max_delta", window=28
    )
    _add_baseline_columns(
        df,
        "mean_PhysicalEffort",
        "physical_effort_baseline_14d",
        "physical_effort_delta",
        window=14,
    )
    _add_baseline_columns(
        df,
        "sleep_duration_hours",
        "sleep_duration_baseline_14d",
        "sleep_duration_delta",
        window=14,
    )
    return df


def _normalize_series(series: pd.Series) -> pd.Series:
    if series.empty or series.dropna().empty:
        return pd.Series(np.nan, index=series.index)
    std = series.std(ddof=0)
    if std == 0 or np.isnan(std):
        return pd.Series(0.0, index=series.index)
    return (series - series.mean()) / std


def compute_readiness_components(daily_df: pd.DataFrame) -> pd.DataFrame:
    """Derive composite readiness score and categorical risk level."""
    if daily_df.empty or "date" not in daily_df.columns:
        return daily_df
    df = daily_df.copy()
    component_weights: list[tuple[str, float]] = []
    df["readiness_component_hrv"] = np.nan
    df["readiness_component_rhr"] = np.nan
    df["readiness_component_sleep"] = np.nan
    if "hrv_delta" in df.columns:
        df["readiness_component_hrv"] = _normalize_series(df["hrv_delta"])
        component_weights.append(("readiness_component_hrv", 0.4))
    if "resting_hr_delta" in df.columns:
        df["readiness_component_rhr"] = -_normalize_series(df["resting_hr_delta"])
        component_weights.append(("readiness_component_rhr", 0.3))
    sleep_basis = None
    if (
        "sleep_efficiency_pct" in df.columns
        and df["sleep_efficiency_pct"].notna().any()
    ):
        sleep_basis = df["sleep_efficiency_pct"]
    elif "sleep_duration_delta" in df.columns:
        sleep_basis = df["sleep_duration_delta"]
    if sleep_basis is not None:
        df["readiness_component_sleep"] = _normalize_series(sleep_basis)
        component_weights.append(("readiness_component_sleep", 0.3))
    if component_weights:
        total_weight = sum(weight for _, weight in component_weights)
        weighted_sum = sum(df[col] * weight for col, weight in component_weights)
        df["readiness_risk_score"] = weighted_sum / total_weight
    else:
        df["readiness_risk_score"] = np.nan
    df["readiness_risk_level"] = pd.cut(
        df["readiness_risk_score"],
        bins=[-np.inf, -0.5, 0.5, np.inf],
        labels=["high", "moderate", "low"],
    )
    return df

In [61]:
# Enrich Apple daily aggregates with derived sleep features (if available)
sleep_features_df = compute_sleep_features(apple_records_df)
if not sleep_features_df.empty:
    apple_daily_df = apple_daily_df.merge(
        sleep_features_df, on="event_date", how="left"
    )
else:
    for col in [
        "sleep_duration_hours",
        "sleep_asleep_hours",
        "sleep_inbed_hours",
        "sleep_sessions",
        "sleep_efficiency_pct",
    ]:
        if col not in apple_daily_df.columns:
            apple_daily_df[col] = np.nan
apple_daily_df = apple_daily_df.sort_values("event_date").reset_index(drop=True)

In [57]:
# Derive glucose daily aggregates directly from the readings DataFrame
glucose_df["timestamp"] = pd.to_datetime(glucose_df["timestamp"]).dt.tz_localize(None)
glucose_df["glucose_rate_change"] = pd.to_numeric(
    glucose_df["glucose_rate_change"], errors="coerce"
)
glucose_df["glucose_range"] = glucose_df["glucose_range"].astype("string").str.lower()

glucose_daily_df = (
    glucose_df.assign(date=glucose_df["timestamp"].dt.floor("D"))
    .groupby("date")
    .agg(
        glucose_mean_mg_dl=("glucose_value", "mean"),
        glucose_median_mg_dl=("glucose_value", "median"),
        glucose_min_mg_dl=("glucose_value", "min"),
        glucose_max_mg_dl=("glucose_value", "max"),
        glucose_std_mg_dl=("glucose_value", "std"),
        glucose_rate_change_avg=("glucose_rate_change", "mean"),
        readings_count=("glucose_value", "count"),
    )
    .reset_index()
)

tir_counts = (
    glucose_df.assign(date=glucose_df["timestamp"].dt.floor("D"))
    .dropna(subset=["glucose_range"])
    .groupby(["date", "glucose_range"])
    .size()
    .unstack(fill_value=0)
)
for col in ["normal", "low", "high", "very_low", "very_high"]:
    if col not in tir_counts.columns:
        tir_counts[col] = 0
tir_pct = tir_counts.div(tir_counts.sum(axis=1), axis=0).mul(100)
tir_pct = tir_pct.rename(
    columns={
        "normal": "tir_normal_pct",
        "low": "tir_low_pct",
        "high": "tir_high_pct",
        "very_low": "tir_very_low_pct",
        "very_high": "tir_very_high_pct",
    }
)

glucose_daily_df = glucose_daily_df.merge(
    tir_pct[[col for col in tir_pct.columns if col.startswith("tir_")]],
    left_on="date",
    right_index=True,
    how="left",
)
glucose_daily_df["date"] = pd.to_datetime(glucose_daily_df["date"])
print(f"📈 Glucose daily aggregates generated: {len(glucose_daily_df):,} days")
display(glucose_daily_df.tail())

📈 Glucose daily aggregates generated: 15 days


,date,glucose_mean_mg_dl,glucose_median_mg_dl,glucose_min_mg_dl,glucose_max_mg_dl,glucose_std_mg_dl,glucose_rate_change_avg,readings_count,tir_low_pct,tir_normal_pct,tir_very_low_pct,tir_high_pct,tir_very_high_pct
10,2025-08-26,84.608,81.0,69.0,126.0,10.576,-0.064,291,0.344,99.656,0.0,0.0,0.0
11,2025-08-27,90.211,87.0,67.0,138.0,15.852,-0.006,289,0.346,99.654,0.0,0.0,0.0
12,2025-08-28,91.212,91.0,73.0,127.0,9.617,-0.007,288,0.000,100.000,0.0,0.0,0.0
13,2025-08-29,91.764,90.0,59.0,132.0,9.679,-0.032,292,1.712,98.288,0.0,0.0,0.0
14,2025-08-30,95.870,92.0,64.0,124.0,9.490,-0.031,215,0.465,99.535,0.0,0.0,0.0


In [58]:
# Build first-pass daily feature mart merging Apple Health and glucose metrics
def normalize_daily_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if df.empty:
        return df
    if "date" not in df.columns:
        for candidate in [
            "event_date",
            "day",
            "start_date",
            "end_date",
            "creationDate",
        ]:
            if candidate in df.columns:
                df["date"] = pd.to_datetime(df[candidate], errors="coerce")
                break
    if "date" not in df.columns:
        raise ValueError(
            "Could not infer a 'date' column for Apple Health daily features."
        )
    date_series = pd.to_datetime(df["date"], errors="coerce")
    if getattr(date_series.dt, "tz", None) is not None:
        date_series = date_series.dt.tz_localize(None)
    df["date"] = date_series
    df = df.drop_duplicates(subset=["date"]).sort_values("date").reset_index(drop=True)
    return df


apple_daily_clean = (
    normalize_daily_columns(apple_daily_df)
    if not apple_daily_df.empty
    else pd.DataFrame()
)
if not apple_daily_clean.empty:
    apple_daily_clean = add_trailing_baselines(apple_daily_clean)
    apple_daily_clean = compute_readiness_components(apple_daily_clean)

if apple_daily_clean.empty and glucose_daily_df.empty:
    daily_feature_mart = pd.DataFrame()
elif apple_daily_clean.empty:
    daily_feature_mart = glucose_daily_df.copy()
elif glucose_daily_df.empty:
    daily_feature_mart = apple_daily_clean.copy()
else:
    daily_feature_mart = (
        apple_daily_clean.merge(
            glucose_daily_df, on="date", how="outer", suffixes=("_apple", "_glucose")
        )
        .sort_values("date")
        .reset_index(drop=True)
    )

print(f"🧩 Daily feature mart shape: {daily_feature_mart.shape}")
display(daily_feature_mart.tail())

🧩 Daily feature mart shape: (15, 159)


,event_date,mean_ActiveEnergyBurned,mean_AppleExerciseTime,mean_AppleStandTime,mean_AppleWalkingSteadiness,mean_BasalEnergyBurned,mean_BloodGlucose,mean_BodyFatPercentage,mean_BodyMass,mean_BodyMassIndex,mean_DietaryCarbohydrates,mean_DietaryCholesterol,mean_DietaryEnergyConsumed,mean_DietaryFatSaturated,mean_DietaryFatTotal,mean_DietaryFiber,mean_DietaryPotassium,mean_DietaryProtein,mean_DietarySodium,mean_DietarySugar,mean_DietaryWater,mean_DistanceCycling,mean_DistanceWalkingRunning,mean_EnvironmentalAudioExposure,mean_EnvironmentalSoundReduction,mean_FlightsClimbed,mean_HeadphoneAudioExposure,mean_HeartRate,mean_HeartRateVariabilitySDNN,mean_LeanBodyMass,mean_PhysicalEffort,mean_RespiratoryRate,mean_RestingHeartRate,mean_SixMinuteWalkTestDistance,mean_StepCount,mean_VO2Max,mean_WalkingAsymmetryPercentage,mean_WalkingDoubleSupportPercentage,mean_WalkingHeartRateAverage,mean_WalkingSpeed,mean_WalkingStepLength,sum_ActiveEnergyBurned,sum_AppleExerciseTime,sum_AppleStandTime,sum_AppleWalkingSteadiness,sum_BasalEnergyBurned,sum_BloodGlucose,sum_BodyFatPercentage,sum_BodyMass,sum_BodyMassIndex,sum_DietaryCarbohydrates,sum_DietaryCholesterol,sum_DietaryEnergyConsumed,sum_DietaryFatSaturated,sum_DietaryFatTotal,sum_DietaryFiber,sum_DietaryPotassium,sum_DietaryProtein,sum_DietarySodium,sum_DietarySugar,sum_DietaryWater,sum_DistanceCycling,sum_DistanceWalkingRunning,sum_EnvironmentalAudioExposure,sum_EnvironmentalSoundReduction,sum_FlightsClimbed,sum_HeadphoneAudioExposure,sum_HeartRate,sum_HeartRateVariabilitySDNN,sum_LeanBodyMass,sum_PhysicalEffort,sum_RespiratoryRate,sum_RestingHeartRate,sum_SixMinuteWalkTestDistance,sum_StepCount,sum_VO2Max,sum_WalkingAsymmetryPercentage,sum_WalkingDoubleSupportPercentage,sum_WalkingHeartRateAverage,sum_WalkingSpeed,sum_WalkingStepLength,count_ActiveEnergyBurned,count_AppleExerciseTime,count_AppleStandTime,count_AppleWalkingSteadiness,count_BasalEnergyBurned,count_BloodGlucose,count_BodyFatPercentage,count_BodyMass,count_BodyMassIndex,count_DietaryCarbohydrates,count_DietaryCholesterol,count_DietaryEnergyConsumed,count_DietaryFatSaturated,count_DietaryFatTotal,count_DietaryFiber,count_DietaryPotassium,count_DietaryProtein,count_DietarySodium,count_DietarySugar,count_DietaryWater,count_DistanceCycling,count_DistanceWalkingRunning,count_EnvironmentalAudioExposure,count_EnvironmentalSoundReduction,count_FlightsClimbed,count_HeadphoneAudioExposure,count_HeartRate,count_HeartRateVariabilitySDNN,count_LeanBodyMass,count_PhysicalEffort,count_RespiratoryRate,count_RestingHeartRate,count_SixMinuteWalkTestDistance,count_StepCount,count_VO2Max,count_WalkingAsymmetryPercentage,count_WalkingDoubleSupportPercentage,count_WalkingHeartRateAverage,count_WalkingSpeed,count_WalkingStepLength,sleep_duration_hours,sleep_asleep_hours,sleep_inbed_hours,sleep_sessions,sleep_efficiency_pct,sleep_onset_time,sleep_wake_time,sleep_midpoint_time,sleep_latency_minutes,sleep_restful_ratio_pct,date,resting_hr_baseline_14d,resting_hr_delta,hrv_baseline_14d,hrv_delta,vo2max_baseline_28d,vo2max_delta,physical_effort_baseline_14d,physical_effort_delta,sleep_duration_baseline_14d,sleep_duration_delta,readiness_component_hrv,readiness_component_rhr,readiness_component_sleep,readiness_risk_score,readiness_risk_level,glucose_mean_mg_dl,glucose_median_mg_dl,glucose_min_mg_dl,glucose_max_mg_dl,glucose_std_mg_dl,glucose_rate_change_avg,readings_count,tir_low_pct,tir_normal_pct,tir_very_low_pct,tir_high_pct,tir_very_high_pct
10,2025-08-26,0.492,1.0,2.097,NaN,1.053,NaN,0.139,151.301,22.371,6.500,7.500,57.500,1.750,3.000,0.500,10.000,2.000,67.500,2.000,NaN,NaN,0.060,51.182,13.809,2.000,67.702,84.692,34.251,130.227,3.581,14.115,57.0,NaN,134.872,NaN,0.005,0.280,101.0,2.672,28.231,1097.437,152.0,151.0,NaN,1812.585,NaN,0.139,151.301,44.743,13.0,7.500,115.0,1.750,6.0,0.500,10.000,2.0,67.500,2.000,NaN,NaN,7.498,2098.471,82.857,10.0,3249.710,203769.180,342.511,130.227,1701.2,7862.0,114.0,NaN,15780.0,NaN,0.09,8.950,101.0,90.842,959.843,2232.0,152.0

In [59]:
# Persist the daily feature mart
DAILY_FEATURE_MART_PATH = PROCESSED_DIR / "feature_mart_daily.parquet"
if daily_feature_mart.empty:
    print("⚠️ Daily feature mart is empty; skipping Parquet export for now.")
else:
    daily_feature_mart.to_parquet(DAILY_FEATURE_MART_PATH, index=False)
    print(f"💾 Saved daily feature mart to {DAILY_FEATURE_MART_PATH}")

💾 Saved daily feature mart to /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/data/processed/feature_mart_daily.parquet


In [60]:
# Event feature mart generation
EVENT_WINDOW_SPEC = {
    "meal": {
        "description": "Identify low-variability glucose windows suitable for meals and track 2h post-event response.",
        "targets": ["glucose_auc_2h", "glucose_peak_2h", "time_above_140_2h_minutes"],
    },
    "cardio": {
        "description": "Exercise blocks linked to next-day recovery (ΔHRV, ΔRHR) and glycemic stability.",
        "targets": [
            "delta_hrv_next_day",
            "delta_rhr_next_day",
            "sleep_efficiency_same_night",
            "next_day_glucose_std_mg_dl",
        ],
    },
    "strength": {
        "description": "Strength sessions balancing effort, next-day readiness, and sleep protection.",
        "targets": ["delta_hrv_next_day", "perceived_effort_balance"],
    },
    "stress": {
        "description": "Readiness risk classification leveraging HRV, RHR, and sleep duration/efficiency.",
        "targets": ["readiness_risk_score"],
    },
}


def build_event_features(
    glucose: pd.DataFrame,
    apple_records: pd.DataFrame,
    windowed_glucose: pd.DataFrame,
    merged_health_glucose: pd.DataFrame,
    daily_feature_mart: pd.DataFrame,
    glucose_daily: pd.DataFrame,
    event_spec: dict[str, dict[str, object]],
) -> pd.DataFrame:
    """Construct event-centric feature mart covering meal, cardio, strength, and stress contexts."""
    events: list[dict[str, object]] = []
    glucose_temp = glucose.copy()
    if not glucose_temp.empty and "timestamp" in glucose_temp.columns:
        glucose_temp["timestamp"] = pd.to_datetime(
            glucose_temp["timestamp"], errors="coerce"
        ).dt.tz_localize(None)
        glucose_temp = glucose_temp.dropna(subset=["timestamp"]).sort_values(
            "timestamp"
        )
    if glucose_temp.empty:
        glucose_series = pd.Series(dtype=float)
        rate_change_series = pd.Series(dtype=float)
    else:
        glucose_series = (
            glucose_temp.set_index("timestamp")["glucose_value"]
            .astype(float)
            .resample("5min")
            .mean()
            .interpolate(limit=12, limit_direction="both")
        )
        if "glucose_rate_change" in glucose_temp.columns:
            rate_change_series = (
                glucose_temp.set_index("timestamp")["glucose_rate_change"]
                .astype(float)
                .resample("5min")
                .mean()
                .interpolate(limit=12, limit_direction="both")
            )
        else:
            rate_change_series = pd.Series(dtype=float)

    def compute_glucose_window_metrics(
        start_ts: pd.Timestamp, hours: int = 2
    ) -> dict[str, float]:
        if pd.isna(start_ts):
            return {
                "pre": np.nan,
                "auc": np.nan,
                "peak": np.nan,
                "std": np.nan,
                "time_above_140": np.nan,
                "mean_rate_change": np.nan,
            }
        start_ts = pd.Timestamp(start_ts).tz_localize(None)
        window_end = start_ts + pd.Timedelta(hours=hours)
        if glucose_series.empty:
            pre_value = np.nan
            window = pd.Series(dtype=float)
        else:
            window = glucose_series.loc[start_ts:window_end]
            pre_value = (
                glucose_series.asof(start_ts) if not glucose_series.empty else np.nan
            )
        if window.empty:
            return {
                "pre": float(pre_value) if pd.notna(pre_value) else np.nan,
                "auc": np.nan,
                "peak": np.nan,
                "std": np.nan,
                "time_above_140": np.nan,
                "mean_rate_change": np.nan,
            }
        dx_hours = 5 / 60
        auc = float(np.trapezoid(window.values, dx=dx_hours))
        peak = float(window.max())
        std_val = float(window.std())
        time_above_140 = float((window > 140).sum() * 5)
        if rate_change_series.empty:
            mean_rate_change = np.nan
        else:
            rate_window = rate_change_series.loc[start_ts:window_end]
            mean_rate_change = (
                float(rate_window.mean()) if not rate_window.empty else np.nan
            )
        return {
            "pre": float(pre_value) if pd.notna(pre_value) else np.nan,
            "auc": auc,
            "peak": peak,
            "std": std_val,
            "time_above_140": time_above_140,
            "mean_rate_change": mean_rate_change,
        }

    def prepare_lookup(df: pd.DataFrame) -> pd.DataFrame:
        if df.empty or "date" not in df.columns:
            return pd.DataFrame()
        lookup = df.copy()
        lookup["date"] = pd.to_datetime(lookup["date"], errors="coerce").dt.floor("D")
        lookup = lookup.dropna(subset=["date"]).drop_duplicates(
            subset=["date"], keep="last"
        )
        return lookup.set_index("date").sort_index()

    daily_lookup = prepare_lookup(daily_feature_mart)
    glucose_daily_lookup = prepare_lookup(glucose_daily)

    def get_metric(lookup: pd.DataFrame, date: pd.Timestamp, column: str) -> float:
        if lookup.empty or column not in lookup.columns or pd.isna(date):
            return np.nan
        key = pd.Timestamp(date).floor("D")
        if key not in lookup.index:
            return np.nan
        value = lookup.at[key, column]
        return float(value) if pd.notna(value) else np.nan

    merged_copy = (
        merged_health_glucose.copy()
        if not merged_health_glucose.empty
        else pd.DataFrame()
    )
    if "meal" in event_spec and not merged_copy.empty:
        meal_mask = merged_copy.get("likely_meal_time")
        if meal_mask is not None:
            if meal_mask.dtype == bool:
                merged_copy = merged_copy[meal_mask]
            else:
                merged_copy = merged_copy[
                    meal_mask.astype(str).str.lower().isin(["true", "1", "yes"])
                ]
        else:
            merged_copy = merged_copy.iloc[0:0]
        if not merged_copy.empty and "glucose_timestamp" in merged_copy.columns:
            merged_copy["glucose_timestamp"] = pd.to_datetime(
                merged_copy["glucose_timestamp"], errors="coerce"
            ).dt.tz_localize(None)
            merged_copy = merged_copy.dropna(subset=["glucose_timestamp"])
            merged_copy = merged_copy.sort_values("glucose_timestamp")
            merged_copy["event_bin"] = merged_copy["glucose_timestamp"].dt.floor(
                "30min"
            )
            meal_candidates = merged_copy.drop_duplicates(subset=["event_bin"])
            for row in meal_candidates.itertuples():
                start_ts = getattr(row, "glucose_timestamp", None)
                metrics = compute_glucose_window_metrics(start_ts)
                events.append(
                    {
                        "event_type": "meal",
                        "event_start": start_ts,
                        "event_end": (
                            start_ts + pd.Timedelta(hours=2)
                            if start_ts is not None
                            else np.nan
                        ),
                        "source": "merged_health_glucose",
                        "pre_glucose_mg_dl": metrics["pre"],
                        "glucose_auc_2h": metrics["auc"],
                        "glucose_peak_2h": metrics["peak"],
                        "time_above_140_2h_minutes": metrics["time_above_140"],
                        "post_window_std_mg_dl": metrics["std"],
                        "post_window_mean_rate_change": metrics["mean_rate_change"],
                        "glucose_trend_label": getattr(row, "glucose_trend", np.nan),
                    }
                )

    workouts = apple_records.copy() if not apple_records.empty else pd.DataFrame()
    if not workouts.empty and {"start_date", "end_date", "type"} <= set(
        workouts.columns
    ):
        for col in ["start_date", "end_date"]:
            workouts[col] = pd.to_datetime(
                workouts[col], errors="coerce"
            ).dt.tz_localize(None)
        workouts = workouts.dropna(subset=["start_date", "end_date"])
        workouts["type_lower"] = workouts["type"].astype(str).str.lower()
        strength_pattern = r"strength|resistance|weight"
        cardio_pattern = r"run|cycle|cardio|aerobic|walk|hiking|swim|row"
        strength_mask = workouts["type_lower"].str.contains(
            strength_pattern, regex=True, na=False
        )
        cardio_mask = (
            workouts["type_lower"].str.contains(cardio_pattern, regex=True, na=False)
            & ~strength_mask
        )

        def create_workout_events(subset: pd.DataFrame, label: str) -> None:
            for row in subset.itertuples():
                start_ts = getattr(row, "start_date", None)
                end_ts = getattr(row, "end_date", None)
                if pd.isna(start_ts) or pd.isna(end_ts):
                    continue
                start_ts = pd.Timestamp(start_ts)
                end_ts = pd.Timestamp(end_ts)
                metrics = compute_glucose_window_metrics(end_ts)
                event_date = start_ts.floor("D")
                next_day = event_date + pd.Timedelta(days=1)
                event_record: dict[str, object] = {
                    "event_type": label,
                    "event_start": start_ts,
                    "event_end": end_ts,
                    "source": getattr(row, "type", np.nan),
                    "duration_minutes": (end_ts - start_ts).total_seconds() / 60,
                    "delta_hrv_next_day": get_metric(
                        daily_lookup, next_day, "hrv_delta"
                    ),
                    "delta_rhr_next_day": get_metric(
                        daily_lookup, next_day, "resting_hr_delta"
                    ),
                    "sleep_efficiency_same_night": get_metric(
                        daily_lookup, next_day, "sleep_efficiency_pct"
                    ),
                    "next_day_glucose_std_mg_dl": get_metric(
                        glucose_daily_lookup, next_day, "glucose_std_mg_dl"
                    ),
                    "glucose_auc_post_2h": metrics["auc"],
                    "glucose_peak_post_2h": metrics["peak"],
                    "glucose_std_post_2h": metrics["std"],
                    "glucose_mean_rate_change_post_2h": metrics["mean_rate_change"],
                    "perceived_effort_balance": np.nan,
                }
                if label == "strength":
                    physical_effort_today = get_metric(
                        daily_lookup, event_date, "mean_PhysicalEffort"
                    )
                    physical_effort_baseline = get_metric(
                        daily_lookup, event_date, "physical_effort_baseline_14d"
                    )
                    if pd.notna(physical_effort_today) and pd.notna(
                        physical_effort_baseline
                    ):
                        event_record["perceived_effort_balance"] = (
                            physical_effort_today - physical_effort_baseline
                        )
                events.append(event_record)

        if cardio_mask.any():
            create_workout_events(workouts[cardio_mask], "cardio")
        if strength_mask.any():
            create_workout_events(workouts[strength_mask], "strength")

    if "stress" in event_spec and not daily_feature_mart.empty:
        stress_df = daily_feature_mart.copy()
        columns_needed = [
            "date",
            "readiness_risk_score",
            "readiness_risk_level",
            "readiness_component_hrv",
            "readiness_component_rhr",
            "readiness_component_sleep",
            "sleep_efficiency_pct",
            "resting_hr_delta",
            "hrv_delta",
        ]
        available_cols = [col for col in columns_needed if col in stress_df.columns]
        stress_df = (
            stress_df[available_cols].dropna(subset=["date"])
            if available_cols
            else pd.DataFrame()
        )
        for row in stress_df.itertuples():
            start_ts = pd.Timestamp(getattr(row, "date")).tz_localize(None)
            events.append(
                {
                    "event_type": "stress",
                    "event_start": start_ts,
                    "event_end": start_ts + pd.Timedelta(days=1),
                    "source": "daily_readiness",
                    "readiness_risk_score": getattr(
                        row, "readiness_risk_score", np.nan
                    ),
                    "readiness_risk_level": str(
                        getattr(row, "readiness_risk_level", "")
                    ),
                    "hrv_delta": getattr(row, "hrv_delta", np.nan),
                    "resting_hr_delta": getattr(row, "resting_hr_delta", np.nan),
                    "sleep_efficiency_pct": getattr(
                        row, "sleep_efficiency_pct", np.nan
                    ),
                    "hrv_component": getattr(row, "readiness_component_hrv", np.nan),
                    "resting_hr_component": getattr(
                        row, "readiness_component_rhr", np.nan
                    ),
                    "sleep_component": getattr(
                        row, "readiness_component_sleep", np.nan
                    ),
                }
            )

    event_df = pd.DataFrame(events)
    if event_df.empty:
        return event_df
    event_df = event_df.sort_values("event_start").reset_index(drop=True)
    event_df["event_id"] = np.arange(1, len(event_df) + 1)
    preferred_order = [
        "event_id",
        "event_type",
        "event_start",
        "event_end",
        "source",
        "pre_glucose_mg_dl",
        "glucose_auc_2h",
        "glucose_peak_2h",
        "time_above_140_2h_minutes",
        "post_window_std_mg_dl",
        "post_window_mean_rate_change",
        "duration_minutes",
        "delta_hrv_next_day",
        "delta_rhr_next_day",
        "sleep_efficiency_same_night",
        "next_day_glucose_std_mg_dl",
        "glucose_auc_post_2h",
        "glucose_peak_post_2h",
        "glucose_std_post_2h",
        "glucose_mean_rate_change_post_2h",
        "perceived_effort_balance",
        "readiness_risk_score",
        "readiness_risk_level",
        "hrv_component",
        "resting_hr_component",
        "sleep_component",
    ]
    existing_cols = [col for col in preferred_order if col in event_df.columns]
    event_df = event_df[
        existing_cols + [col for col in event_df.columns if col not in existing_cols]
    ]
    return event_df


event_feature_mart = build_event_features(
    glucose=glucose_df,
    apple_records=apple_records_df,
    windowed_glucose=windowed_glucose_df,
    merged_health_glucose=merged_health_glucose_df,
    daily_feature_mart=daily_feature_mart,
    glucose_daily=glucose_daily_df,
    event_spec=EVENT_WINDOW_SPEC,
)
display(Markdown("**Event feature mart preview**"))
display(event_feature_mart.head())
EVENT_FEATURE_MART_PATH = PROCESSED_DIR / "feature_mart_events.parquet"
if event_feature_mart.empty:
    print("⚠️ Event feature mart is empty; skipping Parquet export for now.")
else:
    event_feature_mart.to_parquet(EVENT_FEATURE_MART_PATH, index=False)
    print(f"💾 Saved event feature mart to {EVENT_FEATURE_MART_PATH}")

**Event feature mart preview**

,event_id,event_type,event_start,event_end,source,pre_glucose_mg_dl,glucose_auc_2h,glucose_peak_2h,time_above_140_2h_minutes,post_window_std_mg_dl,post_window_mean_rate_change,readiness_risk_score,readiness_risk_level,hrv_component,resting_hr_component,sleep_component,glucose_trend_label,hrv_delta,resting_hr_delta,sleep_efficiency_pct
0,1,stress,2025-08-16 00:00:00,2025-08-17 00:00:00,daily_readiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2,meal,2025-08-16 13:20:00,2025-08-16 15:20:00,merged_health_glucose,127.0,204.500,127.0,0.0,13.058,0.357,NaN,NaN,NaN,NaN,NaN,stable,NaN,NaN,NaN
2,3,meal,2025-08-16 13:30:00,2025-08-16 15:30:00,merged_health_glucose,127.0,202.750,127.0,0.0,11.926,0.247,NaN,NaN,NaN,NaN,NaN,stable,NaN,NaN,NaN
3,4,meal,2025-08-16 14:00:00,2025-08-16 16:00:00,merged_health_glucose,87.0,204.271,117.5,0.0,9.945,-0.020,NaN,NaN,NaN,NaN,NaN,stable,NaN,NaN,NaN
4,5,meal,2025-08-16 14:30:00,2025-08-16 16:30:00,merged_health_glucose,92.0,213.854,117.5,0.0,6.384,-0.028,NaN,NaN,NaN,NaN,NaN,stable,NaN,NaN,NaN


💾 Saved event feature mart to /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/data/processed/feature_mart_events.parquet


### Next steps
- Expand Apple Health daily features with sleep duration/efficiency, VO₂Max trends, and moving baselines
- Implement `build_event_features` for meal, cardio, strength, and stress readiness event windows
- Export event feature mart to `feature_mart_events.parquet` and surface quality checks for downstream Notebook 7